# The Backend
In this next part of the lesson, you'll build a backend using Flask. Because Flask is written in Python, you can use any Python library in your backend including pandas and scikit-learn.

In this part of the lesson, you'll practice:
* setting up the backend
* linking the backend and the frontend together
* deploying the app to a server so that the app is available from a web address

## What is Flask?
[Flask](http://flask.pocoo.org/). A web framework takes care of all the routing needed to organize a web page so that you don't have to write the code yourself!

When you type "http://www.udacity.com" into a browser, your computer sends out a request to another computer (ie the server) where the Udacity website is stored. Then the Udacity server sends you the files needed to render the website in your browser. The Udacity computer is called a server because it "serves" you the files that you requested.

The HTTP part of the web address stands for Hypter-text Transfer Protocol. HTTP defines a standard way of sending and receiving messages over the internet.

When you hit enter in your browser, your computer says "get me the files for the web page at www.udacity.com": except that message is sent to the server with the syntax governed by HTTP. Then the server sends out the files via the protocol as well.

There needs to be some software on the server that can interpret these HTTP requests and send out the correct files. That's where a web framework like Flask comes into play. A framework abstracts the code for receiving requests as well as interpreting the requests and sending out the correct files.

## Why Flask?
* First and foremost, you'll be working with Flask because it is written in Python. You won't need to learn a new programming language.
* Flask is also a relatively simple framework, so it's good for making a small web app.
* Because Flask is written in Python, you can use Flask with any other Python library including pandas, numpy and scikit-learn. In this lesson, you'll be deploying a data dashboard and pandas will help get the data ready.

Continue on to start building the backend.

## Using Flask in the Classroom Workspace
In the next part of the lesson, you'll see a classroom workspace. The classroom workspace already has Flask set up for you. So for now, all you need to do to run the Flask app is to open a Terminal and type.


```shell
python worldbank.py
```

Which executes:
```python
from worldbankapp import app
app.run(host='0.0.0.0', port=1001, debug=True)
```
The `worldbankapp` directory contains a few files:
1. `__init__.py`

```python
from flask import Flask

app = Flask(__name__)

from worldbankapp import routes
```

2. `routes.py`

```python
from worldbankapp import app
from flask import render_template

@app.route('/')
@app.route('index')
def index():
    return render_template('index.html')

@app.route('/project-one')
def project_one():
    return render_template('project_one.html')
```

**and others**

That assumes you are in the default workspace directory within Terminal. That will get the server running.

### Creating New Pages
To create a new web page, you first need to specify the route in the routes.py as well as the name of the html template.

```python
@app.route('/new-route')
def render_the_route():
    return render_template('new_route.html')
```

The route name, function name, and template name do not have to match; however, it's good practice to make them similar so that the code is easier to follow.

The `new_route.html` file must go in the `templates` folder. Flask automatically looks for html files in the `templates` folder.

### What is @app.route?
To use Flask, you don't necessarily need to know what `@app.route` is doing. You only have to remember that the path you place inside of `@app.route()` will be the web address. And then the function you write below `@app.route()` is used to render the correct html template file for the web address.

In Python, the `@` symbol is used for decorators. Decorators are a shorthand way to input a function into another function. Take a look at this code. Python allows you to use a function as an input to another function:
```python
def decorator(input_function):

    return input_function

def input_function():
    print("I am an input function")

decorator_example = decorator(input_function)
decorator_example()
```
Running this code will print the string:
```python
>>> "I am an input function"
```
Decorators provide a short-hand way of getting the same behavior:
```python
def decorator(input_function):
    print("Decorator function")
    return input_function

@decorator
def input_function():
    print("I am an input function")

input_function()
```
This code will print out:
```python
>>> "Decorator function"
>>> "I am an input function"
```

Instead of using a decorator function, you could get the same behavior with the following code:
```python
input_function = decorator(input_function)
input_function()
```

Because `@app.route()` has the `.` symbol, there's an implication that `app` is a class (or an instance of a class) and route is a method of that class. Hence a function written underneath `@app.route()` is going to get passed into the route method. The purpose of `@app.route()` is to make sure the correct web address gets associated with the correct html template. This code:
```python
@app.route('/homepage')
def some_function():
    return render_template('index.html')
```
is ensuring that the web address 'www.website.com/homepage' is associated with the `index.html` template.

If you'd like to know more details about decorators and how `@app.route()` works, check out these tutorials:
* [how @app.route works](https://ains.co/blog/things-which-arent-magic-flask-part-1.html)
* [general decorators tutorial](https://realpython.com/primer-on-python-decorators/)

### Code from the Screencast
Here is the code from the `routes.py` file before refactoring.

The data set comes from this link at the World Bank's data repository: [link to dataset](https://data.worldbank.org/indicator/SP.RUR.TOTL.ZS?view=chart)
```python
from worldbankapp import app
from flask import render_template
import pandas as pd

df = pd.read_csv('data/API_SP.RUR.TOTL.ZS_DS2_en_csv_v2_9948275.csv', skiprows=4)

# Filter for 1990 and 2015, top 10 economies
df = df[['Country Name','1990', '2015']]
countrylist = ['United States', 'China', 'Japan', 'Germany', 'United Kingdom', 'India', 'France', 'Brazil', 'Italy', 'Canada']
df = df[df['Country Name'].isin(countrylist)]

# melt year columns  and convert year to date time
df_melt = df.melt(id_vars='Country Name', value_vars = ['1990', '2015'])
df_melt.columns = ['country','year', 'variable']
df_melt['year'] = df_melt['year'].astype('datetime64[ns]').dt.year

# add column names
df_melt.columns = ['country', 'year', 'percentrural']

# prepare data into x, y lists for plotting
df_melt.sort_values('percentrural', ascending=False, inplace=True)

data = []
for country in countrylist:
    x_val = df_melt[df_melt['country'] == country].year.tolist()
    y_val =  df_melt[df_melt['country'] == country].percentrural.tolist()
    data.append((country, x_val, y_val))
    print(country, x_val, y_val)

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html')

@app.route('/project-one')
def project_one():
    return render_template('project_one.html')
```
### Exercise
The next exercise will be after the section on using Plotly, Pandas, and Flask together. For now, the next part of the lesson has the refactored code shown in this screencast so that you can explore it in more detail. You'll find it in the 2_flask+pandas_example folder.

## Flask with Plotly and Pandas: Pt I
In this next video, you'll see an example of how to pass data from the back end to the front end of the web app. In the next four parts of this lesson, you'll get a sense for how data and Plotly code can be taken from the back end and, sent to the front end, and then used to render plots on the front end. The goal of these next few videos is to show you how the web template works, which you'll be using later in the final exercise.
[video](https://youtu.be/xg7P8MnItdI)

#### Summary Part 1
The purpose of this section is to give you an idea of how the final web app works in terms of passing information back and forth between the back end and front end. The web template you'll be using at the end of the lesson will already provide the code for sharing information between the back and front ends. Your task will be to wrangle data and set up the plotly visualizations using Python. But it's important to get a sense for how the web app works.

In the video above, the data set was sent from the back end to the front end. This was accomplished by including a variable in the render_template() function like so:
```python
data = data_wrangling()

@app.route('/')
@app.route('/')
def index():
    return render_template('index.html', data_set=data)
```

What this code does is to first load the data using the data_wrangling function from wrangling.py. This data gets stored in a variable called data.

In render_template, that data is sent to the front end via a variable called data_set. Now the data is available to the front_end in the data_set variable.

In the index.html file, you can access the data_set variable using the following syntax:
```html
{{ data_set }}
```
You can do this because Flask comes with a template engine called [Jinja](http://jinja.pocoo.org/). [Jinja](http://jinja.pocoo.org/) also allows you to put control flow statements in your html using the following syntax:
```html
{% for tuple in data_set %}
  <p>{{tuple}}</p>
{% end_for %}
```

The logic is:
1. Wrangle data in a file (aka Python module). In this case, the file is called `wrangling.py`. The `wrangling.py` has a function that returns the clean data.
2. Execute this function in `routes.py` to get the data in routes.py
3. Pass the data to the front-end (`index.html` file) using the `render_template()` method.
4. Inside of `index.html`, you can access the data variable with the squiggly bracket syntax `{{ }}`

## Flask with Plotly and Pandas: Pt II
In this section, you'll see how to create a Plotly visualization on the back end and then send the information to the front end for rendering.
[Video](https://youtu.be/yx-DRzMsblI)

### Summary Part 2
In the second part, a Plotly visualization was set up on the back-end inside the `routes.py` file using Plotly's Python library. The Python plotly code is a dictionary of dictionaries. The Python dictionary is then converted to a JSON format and sent to the front-end via the `render_templates()` method.

Simultaneously a list of ids are created for the plots. This information is also sent to the front-end using the `render_template()` method.

On the front-end, the ids and visualization code (JSON code) is then used with the Plotly javascript library to render the plots.

In summary:
1. Python is used to set up a Plotly visualization
2. An id is created associated with each visualization
3. The Python Plotly code is converted to JSON
4. The ids and JSON are sent to the front end (index.html).
5. The front end then uses the ids, JSON, and JavaScript Plotly library to render the plots.

### JavaScript or Python
You could actually do all of this with only JavaScript. You would read the data, wrangle the data, and then create the plots all using JavaScript; however, to do all of this in JavaScript, you'd need to learn more about JavaScript programming. Instead, you can use the pandas and Python skills you already have to wrangle data on the back-end.

## Flask with Plotly and Pandas: Pt III
Here, the screencast video shows how to make more complex visualizations in Plotly. This example shows a line chart containing a unique line for each country in the data set.
[video](https://youtu.be/e8owK5zk-g8)

### Summary Part 3
In part 3, the code iterated through the data set to create a visualization with multiple lines: one for each country.

The original code for a line chart with a single line was:
```python
graph_one = [go.Scatter(
  x = data[0][1],
  y = data[0][2],
  mode = 'lines',
  name = country
)]
```
To make a visualization with multiple lines, graph_one will be a list of line charts. This was accomplished with the following code:
```python
graph_one = []
for data_tuple in data:
   graph_one.append(go.Scatter(
   x = data_tuple[1],
   y = data_tuple[2],
   mode = 'lines',
   name = data_tuple[0]
))
```

### Next
In the last section of flask, plotly, and pandas, you'll see how to add more visualizations to the data dashboard. Then, you'll see some example code and finally you will practice using flask, plotly, and pandas together.

## Flask with Plotly and Pandas: Pt IV
In this next section, you'll see how to add more visualizations in the back end code and then render those visualizations on the front end.
[video](https://youtu.be/4IF2G9Fehb4)

### Summary Part 4
In the last part, three more visualizations were added to the wrangling Python module. The wrangling included reading in the data, cleaning the data, and preparing the Plotly code. Each visualization's code was appended to a list called figures. These visualizations were then imported into the `routes.py` file. This figures list was sent from the back end to the front end via the `render_template()` method. A list of ids were also sent from the back end to the front end.

Then on the front end (`index.html`), a div was created for each visualization's id. And with help from the JavaScript Plotly library, each visualization was rendered inside appropriate div.

#### Beyond a CSV file
Besides storing data in a local csv file (or text, json, etc.), you could also store the data in a database such as a SQL database.

The database could be local to your website meaning that the database file is stored on the same server as your website; alternatively, the database could be stored somewhere else like on a separate database server or with a cloud service like Amazon AWS.

Using a database with your web app goes beyond the scope of this introduction to web development, here are a few resources for using databases with Flask apps:

* [Tutorial - Using Databases with Flask](https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-iv-database)
* [SQL Alchemy- a Python toolkit for working with SQL](http://docs.sqlalchemy.org/en/latest/)
* [Flask SQLAlchemy - a Flask library for using SQLAlchemy with Flask](http://flask-sqlalchemy.pocoo.org/2.3/)

### Next Steps
In the next part of the lesson, you can look at the code and try running the web app from the classroom. Then in the next exercise, you'll practice adding another visualization to the web app.

## Deployment
[video](https://youtu.be/YPfNzpnm_Rk)

### Other Services Besides Heroku
Heroku is just one option of many for deploying a web app, and Heroku is actually owned by [Salesforce.com](https://www.salesforce.com/).

The big internet companies offer similar services like [Amazon's Lightsail](https://aws.amazon.com/lightsail/), [Microsoft's Azure](https://azure.microsoft.com/en-us/resources/samples/python-docs-hello-world/), [Google Cloud](https://cloud.google.com/appengine/docs/standard/python/getting-started/python-standard-env), and [IBM Cloud (formerly IBM Bluemix)](https://www.ibm.com/blogs/bluemix/2015/03/simple-hello-world-python-app-using-flask/). However, these services tend to require more configuration. Most of these also come with either a free tier or a limited free tier that expires after a certain amount of time.

## Instructions Deploying from the Classroom
Here is the code used in the screencast to get the web app running:

First, a new folder was created for the web app and all of the web app folders and files were moved into the folder:
```shell
mkdir web_app
mv -t web_app data worldbankapp wrangling_scripts worldbank.py
```
The next step was to create a virtual environment and then activate the environment:
```shell
conda update python
python3 -m venv worldbankvenv
source worldbankenv/bin/activate
```

Then, pip install the Python libraries needed for the web app
```shell
pip install flask pandas plotly gunicorn
```

The next step was to install the heroku command line tools:
```shell
curl https://cli-assets.heroku.com/install-ubuntu.sh | sh
https://devcenter.heroku.com/articles/heroku-cli#standalone-installation
heroku —-version
```
And then log into heroku with the following command
```shell
heroku login
```
Heroku asks for your account email address and password, which you type into the terminal and press enter.

The next steps involved some housekeeping:

* remove `app.run()` from `worldbank.py`
* type `cd web_app` into the Terminal so that you are inside the folder with your web app code.

Then create a proc file, which tells Heroku what to do when starting your web app:
```shell
touch Procfile
```
Then open the Procfile and type:
```shell
web gunicorn worldbank:app
```
Next, create a requirements file, which lists all of the Python library that your app depends on:
```shell
pip freeze > requirements.txt
```
And initialize a git repository and make a commit:
```shell
git init
git add .
git commit -m ‘first commit’
```
Now, create a heroku app:
```shell
heroku create my-app-name
```
where `my-app-name` is a unique name that nobody else on Heroku has already used.

The `heroku create` command should create a git repository on Heroku and a web address for accessing your web app. You can check that a remote repository was added to your git repository with the following terminal command:
```shell
git remote -v
```
Next, you need to push your git repository to the remote heroku repository with this command:
```shell
git push heroku master
```
Now, you can type your web app's address in the browser to see the results.

### Databases for Your App
The web app in this lesson does not need a database. All of the data is stored in CSV files; however, it is possible to include a database as part of a Flask app. One common use case would be to store user login information such as username and password.

Flask is database agnostic meaning Flask can work with a number of different database types. If you are interested in learning about how to include a database as part of a Flask app, here are some resources:
* [Flask Mega Tutorial](https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-iv-database)
* [Heroku - Provision a Database](https://devcenter.heroku.com/articles/getting-started-with-python#provision-a-database)

### Deployment
In the next part of the lesson, you'll find a workspace where you can practice deploying the world bank web app. Set up an account on [Heroku](www.heroku.com) and then follow the instructions shown in this part of the lesson.

You'll need to use a different name for the web app since the one used in this lesson is already taken.

## ASIDE: Udacity Course Notes Specifics

### Seeing your App in the Workspace
Once the sercer is running, open a new terminal window and type
```shell
env | grep WORK
```
This command will return the Linux environmental variables that contain information about your classroom workspace. The `env` command will list all the environmental variables. The `|` symbol is a pipe for sending output from one command to another. The `grep` command searches text, so `grep WORK` will search for any text containing the word WORK.

The command should return two variables:
```shell
WORKSPACEDOMAIN=udacity-student-workspaces.com
WORKSPACEID=viewc7f3319f2
```

Your WORKSPACEID variable will be different but the WORKSPACEDOMAIN should be the same. Now, open a new web browser window, and type the follwoing in the address bar:
```shell
http://WORKSPACEID-XXXX.WORKSPACEDOMAIN
```

In this example, that would be: https://viewc7f3319f2-3001.udacity-student-workspaces.com/

DON'T FORGET TO INCLUDE `-3001`. You should be able to see the web app. The number 3001 represents the port for accessing your web app.

## Virtual Environments vs. Anaconda
Virtual environments and Anaconda serve a very similar purpose. Anaconda is a distribution of Python (and the analytics language R) specifically for data science. Anaconda comes installed with a package and environment manager called conda. [You can create separate environments using conda](https://conda.io/docs/user-guide/tasks/manage-environments.html). However, these environments automatically come with Python packages meant for data science.

Virtual environments, on the other hand, come with the Python language but do not pre-install other packages.

The classroom workspace has many other Python libraries pre-installed including an installation of [Anaconda](https://www.anaconda.com/distribution/).

When installing a web app to a server, you should only include the packages that are necessary for running your web app. Otherwise you'd be installing Python packages that you don't need.

To ensure that your app only installs necessary packages, you should create a **virtual Python environment**. A virtual Python environment is a separate Python installation on your computer that you can easily remove and won't interfere with your main Python installation.

There is more than one Python package that can set up virtual environments. In the past, you had to install these packages yourself. With Python 3.6, there is a virtual environment package that comes with the Python installation. The packaged is called [venv](https://docs.python.org/3/library/venv.html#module-venv).

However, there is a bug with anaconda's 3.6 Python installation on a Linux system. So in order to use venv in the workspace classroom, you first need to update the Python installation as shown in the instructions above.

### Creating a Virtual Environment Locally on Your Computer
You can develop your app using the classroom workspace. If you decide to develop your app locally on your computer, you should set up a virtual environment there as well. Different versions of Python have different ways of setting up virtual environments. Assuming you are using Python 3.6 and are on a linux or macOS system, then you should be able to set up a virtual environment on your local machine just by typing:
```shell
python3 -m venv name
```
and then to activate:
```shell
source name/bin/activate
```
**On Windows, the command is;**
```shell
c:\>c:\Python35\python -m venv c:\path\to\myenv
```
and to activate:
```shell
C:\> <venv>\Scripts\activate.bat
```
For more information, read through this [link](https://docs.python.org/3/tutorial/venv.html).